<a href="https://colab.research.google.com/github/avish006/Rag-Project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fitz
!pip install --upgrade --force-reinstall pymupdf
!pip install rank_bm25
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━

In [2]:
import fitz
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from openai import OpenAI
from rank_bm25 import BM25Okapi
import nltk
# Download the 'punkt_tab' data package (before using `word_tokenize`)
nltk.download('punkt_tab') # this line has been added
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from langchain.text_splitter import RecursiveCharacterTextSplitter

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import requests  # Use requests to call OpenRouter API
import os


def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF using PyMuPDF."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

In [4]:
def process_pdf(pdf_path):
    text_content = extract_text_from_pdf(pdf_path)

    final_output = f"### Extracted Text from PDF ###\n{text_content}\n\n"

    return final_output

# Example usage
pdf_path = '/content/7 Functional Dependency.ppt.pdf' # Replace with actual PDF file path
text = process_pdf(pdf_path)

In [5]:
#Using Recursive Character Text Splitter for dividing text into chunks of text
def recursive_chunking(text, chunk_size=200, overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_text(text)

In [6]:
#Splitting the documents into chunks of text
chunks = recursive_chunking(text)

In [7]:
#Sentence Transformer for creating vector embeddings of chunks
model = SentenceTransformer('BAAI/bge-large-en-v1.5')
embeddings = [model.encode(chunk,normalize_embeddings=True).tolist() for chunk in chunks]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
#Function to store embeddings
def store_embeddings_faiss(embeddings):
    embedding_dim = len(embeddings[0])  # Get the size of each embedding
    index = faiss.IndexFlatL2(embedding_dim)  # Create FAISS index (L2 norm)

    np_embeddings = np.array(embeddings).astype('float32')  # Convert list to NumPy array
    index.add(np_embeddings)  # Add embeddings to FAISS

    faiss.write_index(index, "vector_store.index")  # Save index to disk
    print("Embeddings stored successfully in FAISS!")

    return index

In [9]:
#Creating an vector index for embeddings
def load_faiss_index():
    return faiss.read_index("vector_store.index")

In [10]:
index = store_embeddings_faiss(embeddings)  # Store embeddings

Embeddings stored successfully in FAISS!


In [11]:
loaded_index = load_faiss_index()  # Load stored embeddings

In [12]:
# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="#####",
)

In [13]:
# Function to retrieve relevant chunks (FAISS example)
def hybrid_search(query, index, chunks, alpha=0.5, top_k=5):

    # Step 1: Vector-based retrieval using FAISS
    query_embedding = model.encode(query,normalize_embeddings=True)  # Ensure this uses the same embedding model used for chunks
    query_embedding = np.array([query_embedding]).astype('float32')

    # Retrieve vector distances and indices for all chunks
    distances, _ = index.search(query_embedding, len(chunks))
    # Convert distances to similarity scores.
    # For L2 distance, a common transformation is similarity = 1 / (1 + distance)
    vector_similarities = [1 / (1 + d) for d in distances[0]]

    # Step 2: Keyword-based retrieval using BM25
    # Tokenize each document chunk (lowercase for uniformity)
    tokenized_chunks = [word_tokenize(chunk.lower()) for chunk in chunks]
    bm25 = BM25Okapi(tokenized_chunks)
    tokenized_query = word_tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)

    # Step 3: Combine scores from vector search and BM25.
    # The combined score is a weighted sum: alpha * BM25 score + (1 - alpha) * Vector similarity.
    combined_scores = []
    for i in range(len(chunks)):
        combined = alpha * bm25_scores[i] + (1 - alpha) * vector_similarities[i]
        combined_scores.append((i, combined))

    # Step 4: Sort the document chunks by the combined score in descending order
    combined_scores.sort(key=lambda x: x[1], reverse=True)

    # Step 5: Get top_k indices and retrieve corresponding document chunks
    top_indices = [idx for idx, score in combined_scores[:top_k]]
    retrieved_chunks = [chunks[i] for i in top_indices]

    return retrieved_chunks

In [14]:
def multi_step_query_rag(query, index, chunks):
    # Step 1: Retrieve top-k relevant chunks from the vector database
    relevant_chunks = hybrid_search(query,index ,chunks)

    # Step 2: Generate a reasoning breakdown
    reasoning_prompt = f"""
    Let's break down this query step by step to enhance reasoning.

    Query: {query}

    Step 1 - Retrieve Relevant Context: {relevant_chunks}

    Step 2 - Identify Key Information: Extract facts, relationships, and key details from the context.

    Step 3 - Apply Logical Deduction: Use the extracted facts to form an answer with reasoning.

    Step 4 - Provide a Final Answer: Give a structured and intuitive response.

    Now, let's perform this step-by-step reasoning.
    """

    # Step 3: Query the LLM with stepwise reasoning
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1:free",
        messages=[
            {"role": "system", "content": "You are an expert in document-based reasoning."},
            {"role": "user", "content": reasoning_prompt}
        ],
        temperature=0.8,  # Increase randomness for more detailed responses
        top_p=0.7 , # Allow for more diverse word choices
        presence_penalty=0.35, #Allow for newer words to be used
        frequency_penalty = 0.2, #Penalizes repeatition of words
        max_tokens= 1600
    )

    return response.choices[0].message.content

In [15]:
# Function to send query to DeepSeek R1 via OpenRouter
def query_rag(user_query, index, chunks):
    reasoning = multi_step_query_rag(user_query, index,chunks)
    retrieved_chunks = hybrid_search(user_query, index, chunks)  # Get top matching chunks

    # Construct full prompt with retrieved context
    full_prompt = f"""
      You are an advanced AI system specializing in deep reasoning and retrieval-augmented generation (RAG). Your goal is to analyze the retrieved context, apply multi-step logical reasoning, and generate a well-structured, insightful response.

      Step 1: Understanding the Query
      User Query:
      {user_query}


      Step 2: Retrieving Relevant Information
      The following context has been retrieved from the document using FAISS vector search:
      {retrieved_chunks}

      Step 3: Multi-Step Reasoning Beyond Retrieval
      To generate the best possible response, follow this structured thought process:

      Direct Extraction (if possible):
      - Identify **explicit** answers in the retrieved context.
      - If the answer is **fully present**, structure it for clarity.

      Inference & Deduction (if required):
      - If the answer is not explicitly stated, use logical inference based on the given context.
      - Identify patterns, relationships, or missing links to construct a complete answer.

      External Knowledge Integration (if needed):
      - If retrieval provides partial data, combine it with general reasoning or background knowledge to improve accuracy.
      - Ensure the information is logically consistent and does not introduce hallucinations.

      Contextual Linking & Deep Reasoning:
      - Connect different retrieved chunks logically to form a complete, well-rounded response.
      - Compare multiple sources, resolve contradictions, and extract the most reliable answer.

      Abstract Interpretation & Implications:
      - If applicable, go beyond factual retrieval to provide a higher-level understanding.
      - Explain why the information matters, potential implications, or how it fits into a broader concept.

      Refer to Reasoning Text:
      - {reasoning}

      Step 4: Generate the Final Answer
      Now, generate a detailed, structured, and logically sound response that follows the above reasoning process.

      Your response should be:
      Comprehensive → Cover key details from the retrieved context.
      Logical → Show the step-by-step reasoning process.
      Insightful → Provide interpretation and implications where necessary.
      Concise but Informative → Avoid unnecessary repetition.

      If uncertainty exists, clearly state the limitations rather than making up information.
      """


    # Send request to DeepSeek R1 via OpenRouter
    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "<YOUR_SITE_URL>",
            "X-Title": "<YOUR_SITE_NAME>",
        },
        model="qwen/qwen2.5-vl-72b-instruct:free",# deepseek/deepseek-r1:free
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.7,  # Increase randomness for more detailed responses
        top_p=0.7 , # Allow for more diverse word choices
        presence_penalty=0.2, #Allow for newer words to be used
        frequency_penalty = 0.2, #Penalizes repeatition of words
        max_tokens= 1600
    )

    return completion.choices[0].message.content

In [17]:
response = query_rag("also tell me about 4nf and 5nf", index, chunks)
print(response)

### **4NF (Fourth Normal Form)**

#### **Purpose**
The primary goal of 4NF is to eliminate redundancy caused by multivalued dependencies (MVDs). This ensures that data is stored in a way that minimizes redundancy and avoids anomalies during updates and queries.

#### **Condition**
A relation is considered to be in 4NF if it meets the following criteria:
1. It is already in BCNF (Boyce-Codd Normal Form).
2. For every non-trivial multivalued dependency \( X \rightarrow\rightarrow Y \), \( X \) must be a superkey.

#### **Example**
Consider the table `company-year(company-id, earnings-2000, earnings-2001, earnings-2002)`. This table is in BCNF but violates 4NF due to the following reasons:
- **Issue**: The table has implicit multivalued dependencies, such as `company-id →→ earnings-year`. This means that for a given `company-id`, the earnings for different years are independent of each other but are all related to the `company-id`.
- **Challenge**: This structure makes querying across dif